In [ ]:
# nb15_reranker_bge.ipynb
# Goal: Implement BGE reranker for improved retrieval precision

# Cell1:  Shared Cache Bootstrap
import os, pathlib, torch
import sys
from datetime import datetime

# Shared cache configuration (複製到每本 notebook)
AI_CACHE_ROOT = os.getenv("AI_CACHE_ROOT", "../ai_warehouse/cache")

for k, v in {
    "HF_HOME": f"{AI_CACHE_ROOT}/hf",
    "TRANSFORMERS_CACHE": f"{AI_CACHE_ROOT}/hf/transformers",
    "HF_DATASETS_CACHE": f"{AI_CACHE_ROOT}/hf/datasets",
    "HUGGINGFACE_HUB_CACHE": f"{AI_CACHE_ROOT}/hf/hub",
    "TORCH_HOME": f"{AI_CACHE_ROOT}/torch",
}.items():
    os.environ[k] = v
    pathlib.Path(v).mkdir(parents=True, exist_ok=True)
print("[Cache]", AI_CACHE_ROOT, "| GPU:", torch.cuda.is_available())

In [ ]:
# Cell 2: Dependencies and Imports
import json
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer, CrossEncoder
from typing import List, Tuple, Dict, Any
import faiss
from pathlib import Path

# Set matplotlib Chinese font
plt.rcParams["font.sans-serif"] = ["SimHei", "DejaVu Sans"]
plt.rcParams["axes.unicode_minus"] = False

print("Dependencies loaded successfully")

In [ ]:
# Cell 3: Load Previous RAG Components
# Load the embedding model and FAISS index from nb13
EMBEDDING_MODEL = "BAAI/bge-m3"
INDEX_PATH = "indices/tech_docs.faiss"
CHUNKS_PATH = "indices/tech_chunks.jsonl"

print("Loading embedding model...")
embedding_model = SentenceTransformer(EMBEDDING_MODEL)
print(f"Embedding model loaded: {EMBEDDING_MODEL}")

# Load FAISS index
if Path(INDEX_PATH).exists():
    index = faiss.read_index(INDEX_PATH)
    print(f"FAISS index loaded: {index.ntotal} vectors")
else:
    print(f"Warning: Index file {INDEX_PATH} not found")
    print("Please run nb13 first to build the index")

# Load chunks metadata
chunks = []
if Path(CHUNKS_PATH).exists():
    with open(CHUNKS_PATH, "r", encoding="utf-8") as f:
        chunks = [json.loads(line) for line in f]
    print(f"Loaded {len(chunks)} chunks")
else:
    print(f"Warning: Chunks file {CHUNKS_PATH} not found")

In [ ]:
# Cell 4: BGE Reranker Setup
RERANKER_MODEL = "BAAI/bge-reranker-base"  # or "BAAI/bge-reranker-large"

print(f"Loading reranker model: {RERANKER_MODEL}")
reranker = CrossEncoder(RERANKER_MODEL)
print("Reranker model loaded successfully")


class BGEReranker:
    def __init__(self, model_name: str = "BAAI/bge-reranker-base"):
        self.reranker = CrossEncoder(model_name)
        self.model_name = model_name

    def rerank(
        self, query: str, passages: List[str], top_k: int = 8
    ) -> List[Tuple[str, float]]:
        """
        Rerank passages based on query relevance
        Returns: List of (passage, score) tuples sorted by relevance
        """
        if not passages:
            return []

        # Create query-passage pairs for reranking
        pairs = [[query, passage] for passage in passages]

        # Get relevance scores
        scores = self.reranker.predict(pairs)

        # Sort by score (higher = more relevant)
        ranked_results = list(zip(passages, scores))
        ranked_results.sort(key=lambda x: x[1], reverse=True)

        return ranked_results[:top_k]


# Initialize reranker
bg_reranker = BGEReranker(RERANKER_MODEL)
print("BGE Reranker initialized")

In [ ]:
# Cell 5: Baseline Retrieval Function
def baseline_retrieval(query: str, k: int = 20) -> List[Dict[str, Any]]:
    """
    Basic vector similarity retrieval without reranking
    """
    # Encode query
    query_vector = embedding_model.encode([query], normalize_embeddings=True).astype(
        "float32"
    )

    # Search FAISS index
    distances, indices = index.search(query_vector, k)

    results = []
    for i, (dist, idx) in enumerate(zip(distances[0], indices[0])):
        if idx < len(chunks):
            chunk = chunks[idx]
            results.append(
                {
                    "rank": i + 1,
                    "text": chunk["text"],
                    "metadata": chunk.get("metadata", {}),
                    "similarity": float(dist),
                    "source": "baseline",
                }
            )

    return results

In [ ]:
# Cell 6: Reranked Retrieval Function
def reranked_retrieval(
    query: str, k: int = 8, oversample: int = 20
) -> List[Dict[str, Any]]:
    """
    Vector retrieval followed by semantic reranking
    """
    # Step 1: Get more candidates than needed (oversample)
    candidates = baseline_retrieval(query, k=oversample)

    # Step 2: Extract passages for reranking
    passages = [result["text"] for result in candidates]

    # Step 3: Rerank using BGE reranker
    reranked_pairs = bg_reranker.rerank(query, passages, top_k=k)

    # Step 4: Map back to original results with rerank scores
    reranked_results = []
    for rank, (passage, rerank_score) in enumerate(reranked_pairs):
        # Find original candidate
        for candidate in candidates:
            if candidate["text"] == passage:
                result = candidate.copy()
                result["rank"] = rank + 1
                result["rerank_score"] = float(rerank_score)
                result["source"] = "reranked"
                reranked_results.append(result)
                break

    return reranked_results

In [ ]:
# Cell 7: Side-by-Side Comparison
def compare_retrieval_methods(query: str, k: int = 5):
    """
    Compare baseline vs reranked retrieval side by side
    """
    print(f"Query: {query}")
    print("=" * 80)

    # Get results from both methods
    baseline_results = baseline_retrieval(query, k=k)
    reranked_results = reranked_retrieval(query, k=k, oversample=20)

    # Display results side by side
    print(f"{'BASELINE RETRIEVAL':<40} | {'RERANKED RETRIEVAL'}")
    print("-" * 80)

    for i in range(k):
        # Baseline result
        if i < len(baseline_results):
            b_result = baseline_results[i]
            b_text = (
                b_result["text"][:60] + "..."
                if len(b_result["text"]) > 60
                else b_result["text"]
            )
            b_score = f"sim:{b_result['similarity']:.3f}"
        else:
            b_text, b_score = "N/A", "N/A"

        # Reranked result
        if i < len(reranked_results):
            r_result = reranked_results[i]
            r_text = (
                r_result["text"][:60] + "..."
                if len(r_result["text"]) > 60
                else r_result["text"]
            )
            r_score = f"rerank:{r_result['rerank_score']:.3f}"
        else:
            r_text, r_score = "N/A", "N/A"

        print(f"{i+1}. {b_text:<35} | {i+1}. {r_text}")
        print(f"   {b_score:<35} |    {r_score}")
        print("-" * 80)

    return baseline_results, reranked_results


# Test comparison
test_query = "什麼是 RAG 檢索增強生成？"
baseline_res, reranked_res = compare_retrieval_methods(test_query, k=5)

In [ ]:
# Cell 8: Batch Evaluation with Multiple Queries
def evaluate_retrieval_methods():
    """
    Evaluate both methods on multiple test queries
    """
    test_queries = [
        "什麼是 RAG 檢索增強生成？",
        "如何優化向量檢索的精度？",
        "FAISS 索引的建立步驟",
        "中文文本分段的最佳實務",
        "嵌入模型的正規化為什麼重要？",
    ]

    results = {
        "query": [],
        "baseline_top1_sim": [],
        "reranked_top1_score": [],
        "baseline_time": [],
        "reranked_time": [],
    }

    for query in test_queries:
        print(f"Evaluating: {query}")

        # Baseline timing
        start_time = time.time()
        baseline_results = baseline_retrieval(query, k=5)
        baseline_time = time.time() - start_time

        # Reranked timing
        start_time = time.time()
        reranked_results = reranked_retrieval(query, k=5, oversample=20)
        reranked_time = time.time() - start_time

        # Record metrics
        results["query"].append(query)
        results["baseline_top1_sim"].append(
            baseline_results[0]["similarity"] if baseline_results else 0.0
        )
        results["reranked_top1_score"].append(
            reranked_results[0]["rerank_score"] if reranked_results else 0.0
        )
        results["baseline_time"].append(baseline_time)
        results["reranked_time"].append(reranked_time)

    return pd.DataFrame(results)


# Run batch evaluation
eval_df = evaluate_retrieval_methods()
print("\nBatch Evaluation Results:")
print(eval_df.to_string(index=False, float_format="%.4f"))

In [ ]:
# Cell 9: Performance Analysis and Visualization
def analyze_performance(eval_df: pd.DataFrame):
    """
    Analyze and visualize performance metrics
    """
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))

    # Plot 1: Top-1 Scores Comparison
    axes[0, 0].plot(
        range(len(eval_df)),
        eval_df["baseline_top1_sim"],
        "o-",
        label="Baseline Similarity",
        color="blue",
    )
    axes[0, 0].plot(
        range(len(eval_df)),
        eval_df["reranked_top1_score"],
        "s-",
        label="Reranked Score",
        color="red",
    )
    axes[0, 0].set_title("Top-1 檢索分數對比")
    axes[0, 0].set_xlabel("查詢編號")
    axes[0, 0].set_ylabel("分數")
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)

    # Plot 2: Latency Comparison
    axes[0, 1].bar(
        ["Baseline", "Reranked"],
        [eval_df["baseline_time"].mean(), eval_df["reranked_time"].mean()],
        color=["blue", "red"],
        alpha=0.7,
    )
    axes[0, 1].set_title("平均檢索延遲對比")
    axes[0, 1].set_ylabel("時間 (秒)")

    # Plot 3: Score Distribution
    axes[1, 0].hist(
        eval_df["baseline_top1_sim"], bins=10, alpha=0.7, label="Baseline", color="blue"
    )
    axes[1, 0].hist(
        eval_df["reranked_top1_score"],
        bins=10,
        alpha=0.7,
        label="Reranked",
        color="red",
    )
    axes[1, 0].set_title("Top-1 分數分布")
    axes[1, 0].set_xlabel("分數")
    axes[1, 0].set_ylabel("頻率")
    axes[1, 0].legend()

    # Plot 4: Latency vs Quality Trade-off
    axes[1, 1].scatter(
        eval_df["baseline_time"],
        eval_df["baseline_top1_sim"],
        c="blue",
        label="Baseline",
        s=100,
        alpha=0.7,
    )
    axes[1, 1].scatter(
        eval_df["reranked_time"],
        eval_df["reranked_top1_score"],
        c="red",
        label="Reranked",
        s=100,
        alpha=0.7,
    )
    axes[1, 1].set_title("延遲 vs 品質權衡")
    axes[1, 1].set_xlabel("延遲 (秒)")
    axes[1, 1].set_ylabel("Top-1 分數")
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig("outs/reranker_performance_analysis.png", dpi=300, bbox_inches="tight")
    plt.show()

    # Summary statistics
    print("\n=== Performance Summary ===")
    print(f"平均 Baseline Top-1 Similarity: {eval_df['baseline_top1_sim'].mean():.4f}")
    print(f"平均 Reranked Top-1 Score: {eval_df['reranked_top1_score'].mean():.4f}")
    print(
        f"分數改善: {eval_df['reranked_top1_score'].mean() - eval_df['baseline_top1_sim'].mean():.4f}"
    )
    print(f"平均 Baseline 延遲: {eval_df['baseline_time'].mean():.4f}s")
    print(f"平均 Reranked 延遲: {eval_df['reranked_time'].mean():.4f}s")
    print(
        f"延遲增加: {eval_df['reranked_time'].mean() - eval_df['baseline_time'].mean():.4f}s"
    )


# Run performance analysis
analyze_performance(eval_df)

In [ ]:
# Cell 10: Save Improved Retriever Configuration
def save_reranker_config():
    """
    Save configuration for the improved retrieval pipeline
    """
    config = {
        "embedding_model": EMBEDDING_MODEL,
        "reranker_model": RERANKER_MODEL,
        "retrieval_params": {"baseline_k": 20, "rerank_k": 8, "oversample_ratio": 2.5},
        "performance_metrics": {
            "avg_baseline_similarity": float(eval_df["baseline_top1_sim"].mean()),
            "avg_reranked_score": float(eval_df["reranked_top1_score"].mean()),
            "avg_baseline_latency": float(eval_df["baseline_time"].mean()),
            "avg_reranked_latency": float(eval_df["reranked_time"].mean()),
        },
    }

    # Ensure outs directory exists
    Path("outs").mkdir(exist_ok=True)

    # Save configuration
    with open("outs/reranker_config.json", "w", encoding="utf-8") as f:
        json.dump(config, f, indent=2, ensure_ascii=False)

    print("Reranker configuration saved to outs/reranker_config.json")
    return config


saved_config = save_reranker_config()

In [ ]:
# Cell 11: Smoke Test
print("=== SMOKE TEST: BGE Reranker ===")

# Test 1: Basic reranker functionality
test_query = "向量檢索"
test_passages = [
    "向量檢索是一種基於向量相似度的資訊檢索技術",
    "今天天氣很好，適合出門走走",
    "FAISS 是 Facebook 開發的向量檢索庫",
    "機器學習模型需要大量的訓練資料",
]

reranked_pairs = bg_reranker.rerank(test_query, test_passages, top_k=3)
print(f"Query: {test_query}")
print("Reranked results:")
for i, (passage, score) in enumerate(reranked_pairs):
    print(f"{i+1}. Score: {score:.4f} | {passage}")

# Test 2: End-to-end retrieval
print(f"\n=== End-to-End Retrieval Test ===")
if chunks:
    test_query = "什麼是嵌入向量正規化？"
    results = reranked_retrieval(test_query, k=3)

    print(f"Query: {test_query}")
    print("Top-3 reranked results:")
    for i, result in enumerate(results):
        text = (
            result["text"][:100] + "..."
            if len(result["text"]) > 100
            else result["text"]
        )
        print(f"{i+1}. Score: {result['rerank_score']:.4f}")
        print(f"   Text: {text}")
        print()

print("✅ Smoke test passed! BGE reranker is working correctly.")

# Final summary
print("\n=== Notebook nb15 完成 ===")
print("✅ BGE Reranker 成功整合")
print("✅ 檢索精度提升量化完成")
print("✅ 效能權衡分析完成")
print("✅ 配置檔案已保存")
print("➡️  準備進行 nb16: Context Optimization")

# Notebook nb15: Reranker BGE 重排機制實作

## Goals（目標）

1. **整合 bge-reranker 模型**：實作語義重排序提升檢索精度
2. **對比評估效果**：量化重排前後的 Recall@5 提升幅度
3. **優化檢索管線**：將重排器嵌入完整 RAG 工作流程
4. **效能權衡分析**：評估重排延遲 vs 精度改善的取捨
5. **可配置化設計**：支援不同重排模型與參數調整

## Notebook Outline（筆記本大綱）

| Cell | 用途 | 內容重點 |
|------|------|----------|
| 1 | Bootstrap | Shared Cache + GPU 檢查 |
| 2 | Dependencies | 安裝 sentence-transformers reranker |
| 3 | Load Previous RAG | 載入 nb13/nb14 的索引與檢索器 |
| 4 | BGE Reranker Setup | 初始化 bge-reranker-base/large |
| 5 | Baseline Retrieval | 無重排的基準檢索結果 |
| 6 | Reranked Retrieval | 加入重排的檢索結果 |
| 7 | Side-by-Side Compare | 對比分析與視覺化 |
| 8 | Batch Evaluation | 多查詢的 Recall@k 評估 |
| 9 | Performance Metrics | 延遲 vs 精度權衡分析 |
| 10 | Smoke Test | 驗證重排器正常運作 |

## Core Code Blocks（核心程式碼）## Smoke Test Cell（煙霧測試）

這個 notebook 的關鍵驗證點：

```python
# 驗證重排器能正確排序相關性
assert len(reranked_pairs) == 3
assert reranked_pairs[0][1] > reranked_pairs[1][1]  # 分數遞減
print("✅ Reranker ordering correct")

# 驗證端到端檢索有改善
if chunks and len(results) > 0:
    assert 'rerank_score' in results[0]
    assert results[0]['rerank_score'] > 0
    print("✅ End-to-end reranked retrieval working")
```

## Key Parameters（關鍵參數）

### Low-VRAM Options（低顯存選項）
```python
# 記憶體受限時使用較小的重排模型
RERANKER_MODEL = "BAAI/bge-reranker-small"  # ~134MB vs base ~560MB

# 批次大小調整
reranker = CrossEncoder(RERANKER_MODEL, max_length=512)  # 限制最大長度

# CPU 回退
if not torch.cuda.is_available():
    RERANKER_MODEL = "BAAI/bge-reranker-small"
    print("Using CPU mode with small reranker model")
```

### Performance Tuning（效能調校）
```python
# 超採樣比例調整（品質 vs 延遲）
oversample_ratios = {
    "fast": 1.5,      # 較少候選，更快
    "balanced": 2.5,  # 預設
    "quality": 4.0    # 更多候選，更精確
}

# 重排批次大小
RERANK_BATCH_SIZE = 32  # 根據 VRAM 調整
```


## When to Use This（使用時機）

**適用場景：**
- 檢索精度比延遲更重要的應用
- 有足夠運算資源進行二次排序
- 需要處理語義相近但字面不同的查詢
- 多語言或專業領域檢索場景

**不適用場景：**
- 極低延遲要求（<100ms）
- 資源受限環境（<4GB VRAM）
- 簡單關鍵字匹配已足夠的場景
- 檢索候選數量很少（<10）

## Stage 2 Progress Update（階段二進度更新）

**已完成：**
- ✅ nb10: 資料載入與清理
- ✅ nb11: 中文分段策略  
- ✅ nb12: BGE-M3 嵌入
- ✅ nb13: FAISS 索引建構
- ✅ nb14: 查詢與引用
- ✅ **nb15: BGE 重排器** ← 當前完成

**核心概念掌握：**
- 語義重排 vs 向量檢索的互補性
- 超採樣策略（oversample）的重要性
- 延遲 vs 精度的權衡分析
- 批次評估與指標量化

**常見陷阱：**
- 重排模型載入記憶體占用較大
- 過小的候選集影響重排效果
- 不同查詢類型的重排效果差異
- 重排分數與向量相似度的尺度不同

**下一步：**
- nb16: Context Optimization（上下文優化）
- nb17: Incremental Update（增量更新）
- nb18: Hybrid Retrieval（混合檢索）
- nb19: Multi-Domain Routing（多域路由）

準備好進行下一個 notebook 了嗎？